# Iowa Liquor Sales Predictive Analysis

- The state of **Iowa** publishes its class **E** license liquor sales data monthly at https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy
- It contains more than $19M$ samples of the detailed transaction records from the liquor vendors to
the various retail stores within the state, since $2012$.
- The dataset records the rich interaction between the $\sim 300$ liquor vendors, 
$\sim 2400$ liquor stores 
(with their lat-long coordinates), selling $\sim 9.4K$ liquor products.

<img src="img/iowa_flag.jpg" height=400 width=400 align=left>

### Scope of this Project

- Analyze the store and product data for retailer Hy-Vee 
- Store segmentation to find valuable store insights
- Market basket analysis
- Sales forecasting to predict future sales

### Housekeeping

In [1]:
import missingno as msno
import numpy as np
import pandas as pd
import re
import sqlite3
import pandas as pd
import gc
from nltk import word_tokenize 
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
# Show all the columns in data frame
pd.set_option("display.max_columns", 120, 'display.max_rows', 500)

### Load the data to a pandas data frame

In [3]:
# Import liquor sales data
df = pd.read_csv('data/Iowa_Liquor_Sales.csv')

In [4]:
# Import product details data
lqdetails = pd.read_csv('data/Iowa_Liquor_Products.csv')

In [5]:
# Removing white spaces from columns
df.columns = df.columns.str.replace(' ','')
lqdetails.columns = lqdetails.columns.str.replace(' ','')

In [6]:
# Filter sales data to Hy-Vee subset
lqsales = df[df['StoreName'].str.contains('Hy-Vee')]

In [7]:
lqsales.head(1)

,Invoice/ItemNumber,Date,StoreNumber,StoreName,Address,City,ZipCode,StoreLocation,CountyNumber,County,Category,CategoryName,VendorNumber,VendorName,ItemNumber,ItemDescription,Pack,BottleVolume(ml),StateBottleCost,StateBottleRetail,BottlesSold,Sale(Dollars),VolumeSold(Liters),VolumeSold(Gallons)
2,INV-28403900139,07/01/2020,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701,NaN,7.0,BLACK HAWK,1012400.0,Irish Whiskies,370.0,PERNOD RICARD USA,15628,Jameson,6,1750,32.48,48.72,2,97.44,3.5,0.92


In [8]:
lqdetails.head(1)

,ItemNumber,CategoryName,ItemDescription,Vendor,VendorName,BottleVolume(ml),Pack,InnerPack,Age,Proof,ListDate,UPC,SCC,StateBottleCost,StateCaseCost,StateBottleRetail,ReportDate
0,901141,Special Order Items,Libertine Absinthe(French Absinthe) Mini,885.0,YAHARA BAY DISTILLERS INC,50,6,1,0,114,04/20/2012,NaN,10892060104050,4.82,28.9,7.23,09/01/2021


In [9]:
# Drop the df dataframe and garbage collect
del [[df]]
gc.collect()

33

### Building the Liquor Sales Database

We will decompose the original dataframe into multiple SQlite tables in order to solve the data inconsistencies
- Products
- Product_Prices, recording the historical bottle price changes of the products
- Vendors
- Stores
- Transactions(master table), which records the vendor number, store number, product (item number), transaction date, bottles sold, volume (either in gallons or in liters), sales amount.

### Products

In [10]:
# Columns to be included in products table
cols = ['Date',
       'Category',
       'CategoryName',
       'ItemNumber',
       'ItemDescription',
       'Pack',
       'BottleVolume(ml)']

product = lqsales[cols].copy()
product['CategoryName'] = product['CategoryName'].str.lower()
product['ItemDescription'] = product['ItemDescription'].str.lower()
product['ItemNumber'] = product['ItemNumber'].apply(str)

In [11]:
# Check for CategoryName and Category association inconsistency
product.groupby(['Category', 'CategoryName']).agg({'Date':'count'}).sort_values(by=['CategoryName'])

,,Date
Category,CategoryName,
1031100.0,100 proof vodka,30284
1022200.0,100% agave tequila,135059
1062300.0,aged dark rum,18891
1082010.0,amaretto - imported,38
1101100.0,american alcohol,7791
1081010.0,american amaretto,19593
1051100.0,american brandies,114739
1071100.0,american cocktails,123759
1081300.0,american cordials & liqueur,111476


In [12]:
# Check for ItemDescription and ItemNumber association inconsistency
product.groupby(['ItemNumber', 'ItemDescription']).agg({'Date':'count'}).sort_values(by=['ItemDescription'])

,,Date
ItemNumber,ItemDescription,
928096,"""beefeater """"24""""""",1
87413,"""jose cuervo silver """"rolling stones""""""",93
258,"""rumchata """"gochatas""""""",1
73058,"""rumchata """"minichatas"""" creamer cups""",590
27081,(ri)1 rye,103
...,...,...
902803,zwack,2
67322,zwack,2
36296,zyr vodka,43


In [13]:
# We will use the most recent CategoryName and ID association to solve the inconsistency
product['Date'] = pd.to_datetime(product['Date'], format='%m/%d/%Y')
product['Year'] = pd.DatetimeIndex(product['Date']).year
category = product.sort_values(['Year']).drop_duplicates(subset = ['Category'], keep = 'last')
temp = product.merge(category[['Category', 'CategoryName']], on = 'Category', how = 'left')
temp.drop(columns = 'CategoryName_x', inplace = True)
temp.rename(columns = {'CategoryName_y': 'CategoryName'}, inplace = True)
product = temp.copy()
product.head(1)

,Date,Category,ItemNumber,ItemDescription,Pack,BottleVolume(ml),Year,CategoryName
0,2020-07-01,1012400.0,15628,jameson,6,1750,2020,irish whiskies


In [14]:
# We will use the most recent ItemDescription and ItemNumber association to solve the inconsistency
item = product.sort_values(['Year']).drop_duplicates(subset = ['ItemNumber'], keep = 'last')
temp = product.merge(item[['ItemNumber', 'ItemDescription']], on = 'ItemNumber', how = 'left')
temp.drop(columns = 'ItemDescription_x', inplace = True)
temp.rename(columns = {'ItemDescription_y': 'ItemDescription'}, inplace = True)
product = temp.copy()
product.head(1)

,Date,Category,ItemNumber,Pack,BottleVolume(ml),Year,CategoryName,ItemDescription
0,2020-07-01,1012400.0,15628,6,1750,2020,irish whiskies,jameson


In [15]:
# We will also use the most commonly recorded bottle volume associated with an item
volume = product.groupby(['ItemNumber']).agg({'BottleVolume(ml)':'value_counts'}).rename(columns = {'BottleVolume(ml)':'count'}).reset_index()
volume['rank'] = volume.groupby(['ItemNumber'])['count'].rank(ascending = False)
volume = volume[volume['rank'] == 1]

In [16]:
volume = product.merge(volume[['ItemNumber', 'BottleVolume(ml)']], on = 'ItemNumber', how = 'left')
volume.drop(columns = 'BottleVolume(ml)_x', inplace = True)
volume.rename(columns = {'BottleVolume(ml)_y': 'BottleVolume'}, inplace = True)
product = volume.copy()
product.head(1)

,Date,Category,ItemNumber,Pack,Year,CategoryName,ItemDescription,BottleVolume
0,2020-07-01,1012400.0,15628,6,2020,irish whiskies,jameson,1750.0


In [17]:
# Cleaned product dataframe
product = product[['ItemNumber', 'ItemDescription', 'Category', 'CategoryName', 'BottleVolume', 'Pack']]
product.drop_duplicates(keep = 'first', inplace = True)
product.shape

(10303, 6)

In [18]:
# Adding proof to the products table
product = product.merge(lqdetails[['ItemNumber', 'Proof']], on = 'ItemNumber', how = 'left')

In [19]:
product.head(1)

,ItemNumber,ItemDescription,Category,CategoryName,BottleVolume,Pack,Proof
0,15628,jameson,1012400.0,irish whiskies,1750.0,6,80.0


In [20]:
# Create new feature "Segment" to broadly categorize the items
product['CategoryName'] = product['CategoryName'].apply(str)
product['Segment'] = np.where(product['CategoryName'].str.contains('rum'),'rum',
                     np.where(product['CategoryName'].str.contains('gin'),'gin', 
                     np.where(product['CategoryName'].str.contains('cocktail'), 'cocktail',
                     np.where(product['CategoryName'].str.contains('brandy|brandies'), 'brandy',
                     np.where(product['CategoryName'].str.contains('whisk|bourbon|scotch'), 'whisky',
                     np.where(product['CategoryName'].str.contains('tequila|mezcal'), 'tequila',
                     np.where(product['CategoryName'].str.contains('schnapps'), 'schnapps',
                     np.where(product['CategoryName'].str.contains('vodka'), 'vodka',
                     np.where(product['CategoryName'].str.contains('spirit'), 'spirit',
                     np.where(product['CategoryName'].str.contains('liqueur'), 'liqueur',
                     np.where(product['CategoryName'].str.contains('brandies'), 'brandy', 'others')))))))))))
product['Segment'].value_counts()

whisky      2475
vodka       1961
others      1701
liqueur     1005
rum          799
tequila      651
brandy       404
cocktail     383
spirit       341
schnapps     316
gin          267
Name: Segment, dtype: int64

In [21]:
# Rename columns for consistency
product.rename(columns = {'ItemNumber':'ProductID', 'ItemDescription':'ProductName', 'Category':'CategoryID'}, inplace = True)
product = product[['ProductID', 'ProductName', 'CategoryID', 'CategoryName', 'Segment', 'BottleVolume', 'Pack', 'Proof']]

In [22]:
product.head(1)

,ProductID,ProductName,CategoryID,CategoryName,Segment,BottleVolume,Pack,Proof
0,15628,jameson,1012400.0,irish whiskies,whisky,1750.0,6,80.0


In [23]:
# CategoryName and Segment are predominantly the more important features and we will therefore leave the null values
product.isnull().sum(axis = 0)

ProductID         0
ProductName       0
CategoryID      223
CategoryName      0
Segment           0
BottleVolume      9
Pack              0
Proof           924
dtype: int64

In [24]:
product.shape

(10303, 8)

### Vendors

In [25]:
# Columns to be included in the vendors table
cols = ['Date',
        'VendorNumber',
        'VendorName']

vendor = lqsales[cols].copy()

In [26]:
vendor['Date'] = pd.to_datetime(vendor['Date'], format='%m/%d/%Y')
vendor['Year'] = pd.DatetimeIndex(vendor['Date']).year

In [27]:
# We will use the most recent VendorName
name = vendor.sort_values(['Year']).drop_duplicates(subset = ['VendorNumber'], keep = 'last')
temp = vendor.merge(name[['VendorNumber', 'VendorName']], on = 'VendorNumber', how = 'left')
temp.drop(columns = 'VendorName_x', inplace = True)
temp.rename(columns = {'VendorName_y': 'VendorName'}, inplace = True)
vendor = temp[['VendorNumber', 'VendorName']]
vendor.drop_duplicates(inplace = True)

In [28]:
# Rename columns to maintain consistency across tables
vendor.rename(columns = {'VendorNumber': 'VendorID'}, inplace = True)

In [29]:
# Check for null values
vendor.isnull().sum(axis = 0)

VendorID      1
VendorName    1
dtype: int64

In [30]:
# Removing nulls as both VendorID/Name is null for the row
vendor.dropna(inplace = True)

In [31]:
vendor.shape

(311, 2)

In [32]:
vendor.head(1)

,VendorID,VendorName
0,370.0,PERNOD RICARD USA


### Stores

In [33]:
# Columns to be included in Stores table
cols = ['StoreNumber',
        'StoreName',
        'City',
        'ZipCode',
        'StoreLocation',
        'CountyNumber',
        'County']

store = lqsales[cols].drop_duplicates(subset = ['StoreNumber'], keep = 'first').copy()
store.dropna(inplace = True)

In [34]:
# Check for null values
store.isnull().sum(axis = 0)

StoreNumber      0
StoreName        0
City             0
ZipCode          0
StoreLocation    0
CountyNumber     0
County           0
dtype: int64

In [35]:
# Standradize the store names
store['StoreName'] = store['StoreName'].apply(lambda x: x.split('/')[0].strip())

def standardize(token):
    token = str(token)
    token = token.replace("``",'')
    token = token.replace('"','')
    token = re.sub("[,`'#.]",'',token)
    token = str(token).lower().replace(' and ',' & ')
    words = word_tokenize(token)
    words = [str(i).capitalize().strip() for i in words]
    words = ' '.join(words)
    words = words.replace('  ',' ')
    return words

for column in ['StoreName', 'City', 'County']:
    store[column] = store[column].apply(standardize)

In [36]:
# Round off ZipCode value and convert to string
store.loc[store['ZipCode'] == '712-2','ZipCode'] = '51529'
store.loc[store['ZipCode'] == '51529']
store['ZipCode'] = store['ZipCode'].apply(int).round().apply(str)

In [37]:
# Format the longitude and lattitude for StoreLocation
def coordinates(loc):
    if str(loc) != 'nan':
        loc = str(loc)
        loc = re.findall(r'[-]\d.*\d', loc)[0]
        lon, lat = loc.split(' ')
        lon = round(float(lon), 2)
        lat = round(float(lat), 2)
        location = [lon, lat]
    else:
        location = np.nan
    return location

store['StoreLocation'] = store['StoreLocation'].apply(coordinates)

In [38]:
# Split the logitude and lattitude coordinates
long = []
lat = []
for i in store['StoreLocation']:
    x, y = str(i).split(', ')
    long.append(float(x.replace('[','')))
    lat.append(float(y.replace(']','')))
    
store['long'] = long
store['lat'] = lat

store.drop('StoreLocation', axis = 1, inplace = True)
store.drop_duplicates(subset = ['StoreNumber'], keep = 'first')

,StoreNumber,StoreName,City,ZipCode,CountyNumber,County,long,lat
9,2663,Hy-vee Food Store,Urbandale,50322,77.0,Polk,-93.74,41.63
18,2647,Hy-vee 7,Cedar Rapids,52411,57.0,Linn,-91.70,42.03
34,2565,Hy-vee Food Store 1636,Spencer,51301,21.0,Clay,-95.15,43.15
61,2662,Hy-vee Wine & Spirits,Muscatine,52761,70.0,Muscatine,-91.04,41.43
64,2555,Hy-vee Food Store,Keokuk,52632,56.0,Lee,-91.40,40.42
87,2571,Hy-vee Food Store 2,Waterloo,50703,7.0,Black Hawk,-92.34,42.53
93,2578,Hy-vee,Charles City,50616,34.0,Floyd,-92.68,43.07
123,2675,Hy-vee 2,Coralville,52241,52.0,Johnson,-91.61,41.72
154,2635,Hy-vee 4,Davenport,52807,82.0,Scott,-90.51,41.57
203,2718,Hy-vee Waukon Dollar Fresh,Waukon,52172,3.0,Allamakee,-91.49,43.26


In [39]:
# Clean up names for consistency across tables
store.rename(columns = {'StoreNumber':'StoreID', 'long':'Long', 'lat':'Lat'}, inplace = True)

In [40]:
store.shape

(163, 8)

In [41]:
store.head(1)

,StoreID,StoreName,City,ZipCode,CountyNumber,County,Long,Lat
9,2663,Hy-vee Food Store,Urbandale,50322,77.0,Polk,-93.74,41.63


### Price

In [42]:
# Columns to be included in the Price table
cols = ['ItemNumber',
        'Date',
        'StateBottleRetail',
        'StateBottleCost']

price = lqsales[cols].copy()
price['Date'] = pd.to_datetime(price['Date'])

In [43]:
# Consider the mean value for StateBottleRetail and StateBottleCost to handle the price variation across a day
price = price.groupby(['ItemNumber', 'Date']).agg({'StateBottleRetail':'mean', 'StateBottleCost':'mean'}).reset_index()

In [44]:
# Clean up names so that it's consistent between tables.
price.rename(columns = {'ItemNumber': 'ProductID'}, inplace = True)

In [45]:
# Check for nulls
price.isnull().sum(axis = 0)

ProductID            0
Date                 0
StateBottleRetail    3
StateBottleCost      3
dtype: int64

In [46]:
# Drop null values
price.dropna(inplace = True)

In [47]:
price.head(1)

,ProductID,Date,StateBottleRetail,StateBottleCost
0,101,2016-10-31,29.97,19.98


### Transactions

In [48]:
# Columns to be included in Transactions table
cols = ['Invoice/ItemNumber',
        'StoreNumber',
        'VendorNumber',
        'ItemNumber',
        'Category',
        'Date',
        'BottlesSold',
        'VolumeSold(Gallons)',
        'Sale(Dollars)']

transaction = lqsales[cols].copy()
transaction['Date'] = pd.to_datetime(transaction['Date'])

In [49]:
# Clean up names for consistency between tables
transaction.rename(columns = {'Invoice/ItemNumber':'TransactionID',
                              'StoreNumber':'StoreID',
                              'VendorNumber':'VendorID',
                              'ItemNumber':'ProductID',
                              'Category':'CategoryID',
                              'VolumeSold(Gallons)':'Volume',
                              'Sale(Dollars)':'Sale'}, inplace = True)

In [50]:
# Checking for null values
# Null values will remain since mapping between CategoryID and ProductID exists in Products table 
# and the Products table itself contains null values for CategoryID
transaction.isnull().sum(axis = 0)

TransactionID       0
StoreID             0
VendorID            1
ProductID           0
CategoryID       5889
Date                0
BottlesSold         0
Volume              0
Sale                7
dtype: int64

In [51]:
transaction.shape

(7235676, 9)

In [52]:
transaction.head(1)

,TransactionID,StoreID,VendorID,ProductID,CategoryID,Date,BottlesSold,Volume,Sale
2,INV-28403900139,2643,370.0,15628,1012400.0,2020-07-01,2,0.92,97.44


In [53]:
transaction.groupby('StoreID').sum('Sale')

,VendorID,CategoryID,BottlesSold,Volume,Sale
StoreID,,,,,
2500,27201273.0,1.076985e+11,881163,2.139135e+05,1.230089e+07
2501,13601253.0,5.323522e+10,587954,1.398733e+05,8.860021e+06
2502,22013001.0,8.718939e+10,1033505,2.647732e+05,1.564264e+07
2503,741058.0,2.985857e+09,21888,5.805100e+03,2.635170e+05
2505,14177283.0,5.690133e+10,490748,1.317897e+05,6.729096e+06
2506,25486261.0,9.992813e+10,916990,2.406103e+05,1.366444e+07
2507,8087379.0,3.184335e+10,266394,5.996225e+04,2.728390e+06
2508,17306112.0,6.880599e+10,673161,1.695607e+05,8.967571e+06
2509,12205262.0,4.799495e+10,560677,1.290344e+05,5.795250e+06


### Sales

In [54]:
# Creating a sales table for analyzing store sales later
cols = ['ItemNumber',
        'ItemDescription',
        'Date',
        'Sale(Dollars)',
        'BottlesSold',
        'StoreNumber', 
        'StoreName',
        'CategoryName',
        'BottleVolume(ml)',
        'VendorNumber', 
        'VendorName',
        'County']

dfsales = lqsales[cols].copy()
dfsales['Date'] = pd.to_datetime(dfsales['Date'])

cols = ['ItemNumber',
        'Age', 
        'Proof']
dfitem = lqdetails[cols].copy()

cols = ['StoreID',
        'Long', 
        'Lat']
dfstore = store[cols].copy().rename(columns = {'StoreID':'StoreNumber'})

In [55]:
dfsales['ItemNumber'] = dfsales['ItemNumber'].apply(str)
dfitem['ItemNumber'] = dfitem['ItemNumber'].apply(str)
temp = dfsales.merge(dfitem[['ItemNumber', 'Age', 'Proof']], on = 'ItemNumber', how = 'left')

In [56]:
temp.head(1)

,ItemNumber,ItemDescription,Date,Sale(Dollars),BottlesSold,StoreNumber,StoreName,CategoryName,BottleVolume(ml),VendorNumber,VendorName,County,Age,Proof
0,15628,Jameson,2020-07-01,97.44,2,2643,Hy-Vee Wine and Spirits / Waterloo,Irish Whiskies,1750,370.0,PERNOD RICARD USA,BLACK HAWK,0.0,80.0


In [57]:
dfstore['StoreNumber'] = dfstore['StoreNumber'].apply(str)
temp['StoreNumber'] = temp['StoreNumber'].apply(str)
df = temp.merge(dfstore[['StoreNumber', 'Long', 'Lat']], on = 'StoreNumber', how = 'left')

In [58]:
df.head(1)

,ItemNumber,ItemDescription,Date,Sale(Dollars),BottlesSold,StoreNumber,StoreName,CategoryName,BottleVolume(ml),VendorNumber,VendorName,County,Age,Proof,Long,Lat
0,15628,Jameson,2020-07-01,97.44,2,2643,Hy-Vee Wine and Spirits / Waterloo,Irish Whiskies,1750,370.0,PERNOD RICARD USA,BLACK HAWK,0.0,80.0,NaN,NaN


In [59]:
sales = df.copy()
sales = sales.rename(columns = {'ItemNumber':'ProductID', 'ItemDescription':'ProductName', 'Sale(Dollars)':'Sale',  
                        'StoreNumber':'StoreID', 'BottleVolume(ml)':'BottleVolume', 'VendorNumber':'VendorID'})

### The SQLite Database

In [60]:
# Setting up the database connection
conn = sqlite3.connect('IowaLiquorSales.db')
cursor = conn.cursor()

In [61]:
# Create the tables
cursor.execute('''CREATE TABLE Product
                (ProductID, ProductName, CategoryID, CategoryName, Segment, BottleVolume, Pack, Proof)''')

cursor.execute('''CREATE TABLE Price
                (ProductID, Date, StateBottleRetail, StateBottleCost)''')

cursor.execute('''CREATE TABLE Vendor
                (VendorID, VendorName)''')

cursor.execute('''CREATE TABLE Store
                (StoreID, StoreName, City, ZipCode, CountyNumber, County, Long, Lat)''')

cursor.execute('''CREATE TABLE Transactions
                (TransactionID, StoreID, VendorID, ProductID, CategoryID, Date, BottlesSold, Volume, Sale)''')

cursor.execute('''CREATE TABLE Sales
                (ProductID, ProductName, Date, Sale, BottlesSold, StoreID, StoreName, CategoryName, BottleVolume,
                VendorID, VendorName, County, Age, Proof, Long, Lat)''')

In [62]:
product.to_sql('Product', conn, if_exists = 'append', index = False)
price.to_sql('Price', conn, if_exists = 'append', index = False)
vendor.to_sql('Vendor', conn, if_exists = 'append', index = False)
store.to_sql('Store', conn, if_exists = 'append', index = False)
transaction.to_sql('Transactions', conn, if_exists = 'append', index = False)
sales.to_sql('Sales', conn, if_exists = 'append', index = False)

In [63]:
conn.commit()
conn.close()